In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from gensim.models import Word2Vec

# Load Data

In [3]:
df = pd.read_csv("../datasets/netflix_titles.csv")
df.head(5)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...


# Data Cleaning

In [4]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer


def data_cleaning(text):
    # make lower case
    text = text.lower()
    
    # remove stopwords
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    
    # remove punctuation
    tokenizer = RegexpTokenizer(r'[a-zA-z]+')
    text = tokenizer.tokenize(text)
    text = " ".join(text)
    
    
    return text

In [5]:
df['cleaned'] = df['description'].apply(data_cleaning)
df['cleaned'][:5]

0    future elite inhabit island paradise far crowd...
1    devastating earthquake hits mexico city trappe...
2    army recruit found dead fellow soldiers forced...
3    postapocalyptic world rag doll robots hide fea...
4    brilliant group students become card counting ...
Name: cleaned, dtype: object

In [6]:
corpus = []
for words in df['cleaned']:
    corpus.append(words.split())

# Word Embedding (with pretrained embedded word)

In [6]:
import urllib.request
urllib.request.urlretrieve("https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz", filename="GoogleNews-vectors-negative300.bin.gz")

('GoogleNews-vectors-negative300.bin.gz',
 <http.client.HTTPMessage at 0x1ee96e10670>)

In [7]:
word2vec_model = Word2Vec(size = 300, window=5, min_count = 2, workers = -1)
word2vec_model.build_vocab(corpus)
word2vec_model.intersect_word2vec_format('GoogleNews-vectors-negative300.bin.gz', lockf=1.0, binary=True)
word2vec_model.train(corpus, total_examples = word2vec_model.corpus_count, epochs = 15)

(0, 0)

# Average of word vectors

In [8]:
def vectors(document_list):
    document_embedding_list = []
    
    for line in document_list:
        doc2vec = None
        count = 0
        for word in line.split():
            if word in word2vec_model.wv.vocab:
                count+=1
                if doc2vec is None:
                    doc2vec = word2vec_model[word]
                else:
                    doc2vec = doc2vec + word2vec_model[word]
        if doc2vec is not None:
            doc2vec = doc2vec / count
            document_embedding_list.append(doc2vec)

    return document_embedding_list

In [9]:
document_embedding_list = vectors(df['cleaned'])
print('number of vector',len(document_embedding_list))

<ipython-input-8-d2dc40099e50>:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  doc2vec = word2vec_model[word]
<ipython-input-8-d2dc40099e50>:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  doc2vec = doc2vec + word2vec_model[word]


number of vector 7787


In [11]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarities = cosine_similarity(document_embedding_list, document_embedding_list)
print(cosine_similarities)

[[1.0000001  0.5550802  0.6356698  ... 0.46868804 0.5077918  0.3826328 ]
 [0.5550802  1.0000002  0.5706528  ... 0.482546   0.34191734 0.33678618]
 [0.6356698  0.5706528  1.0000002  ... 0.4384097  0.45118934 0.4293788 ]
 ...
 [0.46868795 0.48254597 0.43840984 ... 1.0000004  0.3883486  0.5431248 ]
 [0.5077918  0.34191734 0.45118934 ... 0.3883486  1.0000002  0.40383375]
 [0.3826328  0.33678618 0.4293788  ... 0.5431248  0.40383375 1.0000002 ]]


In [36]:
def recommendation(title):
    show = df[['title','listed_in','description']]
    
    indices = pd.Series(df.index, index = df['title']).drop_duplicates()
    idx = indices[title]
    
    sim_scores = list(enumerate(cosine_similarities[idx]))
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    
    show_indices = [i[0] for i in sim_scores]
    
    recommend = show.iloc[show_indices].reset_index(drop=True)
    print('Recommended list')
    recommend_df = pd.DataFrame(recommend)
    print(dtype(recommend_df))

In [37]:
recommendation("3%")

Recommended list


NameError: name 'dtype' is not defined